In [1]:
import requests
import json
import pandas as pd

# function to get wind direction based on degrees
def get_wind_direction(degrees):
    direction_names = ["N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE", "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW"]
    index = round(degrees / (360. / len(direction_names))) % len(direction_names)
    return direction_names[index]

# URL to access weather data. Add this from your json packet when inspecting the source. 
url = 'https://################'

# station ID to get weather data for. You will need to add your own station
station_id = 'IFLETC15'

# start and end dates to get weather data for
start_date = '20230205'
end_date = '20230314'

# API key to access weather data. You will need to add your own api key
api_key = '############################'

# create a list of dates to get weather data for
dates = pd.date_range(start=start_date, end=end_date, freq='D')

# create an empty list to store all the weather data
all_data = []

# loop through all the dates and get weather data for each date
for date in dates:
    # set the parameters for the API request
    params = {
        'stationId': station_id,
        'format': 'json',
        'units': 'm',
        'date': date.strftime('%Y%m%d'),
        'numericPrecision': 'decimal',
        'apiKey': api_key
    }
    
    # send the API request and get the response
    response = requests.get(url, params=params)

    # check if the API request was successful
    if response.status_code == 200:
        # get the weather data from the response
        data = json.loads(response.text)["observations"]
        # loop through all the rows in the weather data and add wind direction data
        for row in data:
            metric = row.pop("metric")
            row.update(metric)
            row["wind_direction"] = get_wind_direction(row["winddirAvg"])

        # add the weather data to the list of all data
        all_data += data
    else:
        # print an error message if the API request failed
        print(f'Request failed with status code {response.status_code}')
        
# create a Pandas DataFrame from the weather data and save it to a CSV file
df = pd.DataFrame.from_dict(all_data)
df.to_csv('weather_data1.csv', index=False)
